In [ ]:
!pip install transformers datasets
!pip install transformers[torch]
!pip install accelerate -U
!pip install transformers datasets evaluate

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset
xray=load_dataset("keremberke/chest-xray-classification" ,"full")


# Dataset Loading

A dataset named xray is loaded, containing training, validation, and test sets.
The dataset seems to include image file paths, actual images, and corresponding labels (0 for 'NORMAL' and 1 for 'PNEUMONIA').

In [ ]:
xray

In [ ]:
xray = xray.remove_columns("image_file_path")

In [ ]:
xray["train"][10]

# Data preprocessing

Image file paths are removed from the dataset.
Labels and their corresponding IDs are extracted and stored in dictionaries (label2id and id2label).

In [ ]:
#xray
labels = xray["train"].features["labels"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [ ]:
id2label

# Image Preprocessing

An image processor is loaded using the Hugging Face Transformers library (AutoImageProcessor).
Preprocessing transformations (e.g., resizing, normalization) are defined using the torchvision.transforms module.

In [ ]:
from transformers import AutoImageProcessor

checkpoint = "microsoft/resnet-50"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [ ]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [ ]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

In [ ]:
xray = xray.with_transform(transforms)

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# Model Training Configuration:

The training configuration is set using the TrainingArguments class, specifying parameters like output directory, learning rate, batch size, etc.
An image classification model is loaded using the Hugging Face Transformers library (AutoModelForImageClassification).

In [ ]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

In [ ]:
training_args = TrainingArguments(
    output_dir="Cheese_xray",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=xray["train"],
    eval_dataset=xray["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
# Evaluate the model on the validation set
results = trainer.evaluate()

# Print the evaluation results
print(results)


In [ ]:
xray

In [ ]:
print(xray["validation"].features)


In [ ]:
# Assuming your model is a ResNet model
# You may need to adjust this based on your actual model architecture
last_conv_layer = None

for name, module in model.named_modules():
    if isinstance(module, torch.nn.modules.conv.Conv2d):
        last_conv_layer = module

# Check if a valid last convolutional layer was found
if last_conv_layer is not None:
    print("Found last convolutional layer:", last_conv_layer)
else:
    print("No valid last convolutional layer found in the model.")



In [ ]:
#load model from the last checkpoint
model = AutoModelForImageClassification.from_pretrained(
    "barghavani/Cheese_xray",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)


In [ ]:
trainer.push_to_hub()

In [ ]:
# Download unseen data to check the accuracyy of model
ds = load_dataset("keremberke/chest-xray-classification", "full", split="validation")

image = ds["image"][0]
image

In [ ]:
from transformers import pipeline
classifier = pipeline("image-classification", model="barghavani/Cheese_xray")
classifier(image)


by 66% it predict to be normal and by 34% it is PNEUMONIA

In [ ]:
from transformers import AutoImageProcessor
import torch

image_processor = AutoImageProcessor.from_pretrained("barghavani/Cheese_xray")
inputs = image_processor(image, return_tensors="pt")

In [ ]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained("barghavani/Cheese_xray")
with torch.no_grad():
    logits = model(**inputs).logits

In [ ]:
predicted_label = logits.argmax(-1).item()
model.config.id2label[predicted_label]

# Demo

https://huggingface.co/spaces/barghavani/barghavani-Cheese_xray